### Train model

In [19]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import requests
import datetime
import torch
import torch.utils.data
import os
import importlib
import sys
import re
import pickle
from mpl_toolkits import mplot3d
from io import BytesIO
from math import log, exp, tan, atan, ceil
from PIL import Image

from utils import dataset_utils
from utils import createAISdata
#from utils import protobufDecoder
#from utils import plotting
from models import VRNN
from Config import config

# To measure the training time
from time import time

In [20]:
#path = "C://Users//asm//OneDrive - Netcompany/University//Master Thesis//pickleFiles//"
#path_index = "C://Users//asm//OneDrive - Netcompany/University//Master Thesis//pickleFiles//CargTank_idxs.pkl"

path = "C://Users//asm//OneDrive - Netcompany/University//Master Thesis//Data//Pickle//"
path_index = "C://Users//asm//OneDrive - Netcompany/University//Master Thesis//Data//Pickle//CargTank_dxs.pkl"


datasets_path = "C://Users//asm//OneDrive - Netcompany/University//Master Thesis//Data//Pickle//"
#datasets_path = "C://Users//asm//OneDrive - Netcompany/University//Master Thesis//Data//Pickle//"

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f">> Using device: {device}")
if device=="cuda:0":
    torch.no_grad()
    torch.cuda.empty_cache()

>> Using device: cpu


In [22]:
shiptypes = config.SHIPTYPE_CARGO + config.SHIPTYPE_TANKER
shipFileName = 'test'
binedges = (config.LAT_EDGES, config.LON_EDGES, config.SOG_EDGES, config.COG_EDGES)
batch_size = 4

In [23]:
class PadSequence:
    def __call__(self, batch):
                
        # each element in "batch" is a tuple ( mmsis,  shiptypes,  lengths, inputs, targets)
        # Get each sequence and pad it
        mmsis = [x[0] for x in batch] # Maritime Mobile Service Identity numbers
        shiptypes = [x[1] for x in batch] # tank, cargo, etc.
        speedBinLabel = [x[2] for x in batch] # tank, cargo, etc.
        lengths = [x[3] for x in batch] # used as measure of size
        inputs = [x[4] for x in batch] # they are normalized 
        targets = [x[5] for x in batch] # seems to contain the real path of the vessel
                                        # lat, lon, speed, course (NOT NORMALIZED)
                
        inputs_padded = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True)
        targets_padded = torch.nn.utils.rnn.pad_sequence(targets, batch_first=True)

        return  torch.tensor(mmsis),  torch.tensor(shiptypes),torch.tensor(speedBinLabel),  torch.tensor(lengths, dtype=torch.float), inputs_padded, targets_padded


In [24]:
trainset = dataset_utils.AISDataset(dataPath = path, fileName = "CargTank.pkl")
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers = 0, collate_fn=PadSequence())

testset = dataset_utils.AISDataset(dataPath = path, fileName = "CargTank.pkl", train_mean = trainset.mean)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers = 0, collate_fn=PadSequence())

dataPath: C://Users//asm//OneDrive - Netcompany/University//Master Thesis//Data//Pickle//
fileName: CargTank.pkl
self.params[dataFileName]: CargTank_dxs.pkl


self.datapath 12 C://Users//asm//OneDrive - Netcompany/University//Master Thesis//Data//Pickle//CargTank_dxs.pkl
index:  22787567   total_updates:  643429
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(st

convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(

convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(t

convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(t

...

convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(

convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str

convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(tr

convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(t

convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Tanker
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str(track[shiptype]))  Cargo
convertShipTypeToName(str

In [25]:
train_n = len(trainset)
test_n = len(testset)
num_batches = len(train_loader)
num_epochs = 1

In [26]:
model = VRNN.VRNN(input_shape=trainset.datadim, latent_shape=config.LATENT_SIZE, generative_bias=trainset.mean, device=device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)

In [27]:
def computeLoss(log_px, log_pz, log_qz, lengths, beta=1):
    
    max_len = len(log_px)
    curmask = torch.arange(max_len, device=device)[:, None] < lengths[None, :] #max_seq_len X Batch
    
    log_px = torch.stack(log_px, dim=0) * curmask
    log_px = log_px.sum(dim=0) #Sum over time
   
    log_pz = torch.stack(log_pz, dim=0) * curmask
    log_qz = torch.stack(log_qz, dim=0) * curmask
    kl = log_qz.sum(dim=0) - log_pz.sum(dim=0) #Sum over time
    
    loss = log_px - beta * kl #recon loss - beta_kl
    loss = torch.mean(loss/lengths) #mean over batch
    
    return -loss, log_px, kl

In [28]:
loss_tot = []
kl_tot = []
recon_tot = []
val_loss_tot = []
val_kl_tot = []
val_recon_tot = []

for epoch in range(1, num_epochs+1):
    
    #num_epochs+1
    #Begin training loop
    tic = time()

    loss_epoch = 0
    kl_epoch = 0
    recon_epoch = 0
    model.train()
    
    #for i, (mmsi, label,speedBinLabel,courseBinLabel, lengths, inputs, targets) in enumerate(train_loader):
    for i, (mmsi, label,speedBinLabel, lengths, inputs, targets) in enumerate(train_loader):
    #for i, (mmsi, label,speedBinLabel, lengths, inputs, targets) in enumerate(train_loader):
            
        inputs = inputs.to(device)
        targets = targets.to(device)
        lengths = lengths.to(device)
        
        print('label: ', label)
        
        #log_px, log_pz, log_qz, _, _, z_mus = model(inputs,targets,logits=None)
        log_px, log_pz, log_qz, _, _, z_mus = model(inputs,targets,label,logits=None)
        
        loss, log_px, kl = computeLoss(log_px, log_pz, log_qz, lengths)
        
        model.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_epoch += loss.item()*len(lengths)
        kl_epoch += torch.sum(kl/lengths).item()
        recon_epoch += torch.sum(log_px/lengths).item()
        
        #z_means = {
        #    'Epoch': epoch,
        #    'z_means': z_mus,
        #    'labels': label,
        #    'log_px': log_px,
        #    'log_pz': log_pz,
        #    'log_qz': log_qz
        #    }


        
        print('inputs',inputs.shape)
        print('targets',targets.shape)
        print('lengths',lengths.shape)
        #print('speedBinLabel',speedBinLabel)
        #print('courseBinLabel',courseBinLabel)
        
        break;


label:  tensor([0, 0, 1, 1])
inputs torch.Size([4, 72, 167])
targets torch.Size([4, 72, 167])
lengths torch.Size([4])
